In [43]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [44]:
cats_path = "C:/projects/classifier/backend/PetImages/Cat"
dogs_path = "C:/projects/classifier/backend/PetImages/Dog"

cat_files = [os.path.join(cats_path, file) for file in os.listdir(cats_path)]
dog_files = [os.path.join(dogs_path, file) for file in os.listdir(dogs_path)]

In [45]:
all_files = cat_files + dog_files
labels = [0] * len(cat_files) + [1] * len(dog_files)

In [46]:
train_files, temp_files, train_labels, temp_labels = train_test_split(
    all_files, labels, test_size=0.3, random_state=42
)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42
)

In [12]:
print(f"Train set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")
print(f"Test set: {len(test_files)} images")

Train set: 17472 images
Validation set: 3744 images
Test set: 3745 images


In [14]:
def load_images(file_paths, labels, target_size=(128, 128)):
    images = []
    valid_labels = []
    for file, label in zip(file_paths, labels):
        try:
            img = tf.keras.preprocessing.image.load_img(file, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            valid_labels.append(label)
        except Exception as e:
            print(f"Skipping file {file}: {e}")
    return np.array(images), np.array(valid_labels)

train_images, train_labels = load_images(train_files, train_labels)
val_images, val_labels = load_images(val_files, val_labels)
test_images, test_labels = load_images(test_files, test_labels)

Skipping file C:/projects/classifier/backend/PetImages/Dog\Thumbs.db: cannot identify image file <_io.BytesIO object at 0x000001C25C589E90>


c:\Users\sushe\.conda\envs\deeplearning\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Skipping file C:/projects/classifier/backend/PetImages/Cat\Thumbs.db: cannot identify image file <_io.BytesIO object at 0x000001C25C60E0C0>


In [15]:
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [22]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [23]:
history = model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=10,
    batch_size=32
)

Epoch 1/10
546/546 [==============================] - 204s 370ms/step - loss: 0.6113 - accuracy: 0.6484 - val_loss: 0.5538 - val_accuracy: 0.7145
Epoch 2/10
546/546 [==============================] - 200s 365ms/step - loss: 0.5019 - accuracy: 0.7605 - val_loss: 0.4771 - val_accuracy: 0.7714
Epoch 3/10
546/546 [==============================] - 202s 370ms/step - loss: 0.4319 - accuracy: 0.8020 - val_loss: 0.4201 - val_accuracy: 0.8106
Epoch 4/10
546/546 [==============================] - 197s 361ms/step - loss: 0.3719 - accuracy: 0.8322 - val_loss: 0.4121 - val_accuracy: 0.8200
Epoch 5/10
546/546 [==============================] - 197s 361ms/step - loss: 0.3139 - accuracy: 0.8676 - val_loss: 0.3695 - val_accuracy: 0.8454
Epoch 6/10
546/546 [==============================] - 198s 362ms/step - loss: 0.2612 - accuracy: 0.8891 - val_loss: 0.3642 - val_accuracy: 0.8486
Epoch 7/10
546/546 [==============================] - 196s 359ms/step - loss: 0.2088 - accuracy: 0.9156 - val_loss: 0.3718 -

In [24]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

117/117 - 9s - loss: 0.4789 - accuracy: 0.8627 - 9s/epoch - 76ms/step
Test Accuracy: 0.86


In [26]:
model.save('C:/projects/classifier/backend/cat_dog_classifier.h5')
print("Model saved as cat_dog_classifier.h5")

Model saved as cat_dog_classifier.h5


In [4]:
from tensorflow.keras.models import load_model
import cv2

In [3]:
MODEL_PATH = 'C:/projects/classifier/backend/cat_dog_classifier.h5'  
model = load_model(MODEL_PATH)

In [38]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array = img_array / 255.0 
    return np.expand_dims(img_array, axis=0)  

image_path = ""
image = preprocess_image(image_path)

In [39]:
prediction = model.predict(image)

1/1 [==============================] - 0s 59ms/step


In [40]:
if prediction[0]>0.5:
    print("DOG")
else:
    print("CAT")

CAT


In [42]:
print(prediction)

[[0.14449923]]
